In [ ]:
from flask import Flask, jsonify, render_template
import psycopg2
from psycopg2.extras import RealDictCursor
from config import db_name, user, password, host
import requests

app = Flask(__name__)

# Function to establish a database connection
def get_db_connection():
    conn = psycopg2.connect(
        dbname=db_name,
        user=user,
        password=password,
        host=host
    )
    return conn

# Function to fetch landslide data from PostgreSQL
@app.route('/map', methods=['GET'])
def get_landslide_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('''
        SELECT
            cod_reg,
            cod_rip,
            COD_PROV,
            ar_kmq,
            nome,
            uid,
            ar_fr_p3p4,
            ar_fr_p4,
            ar_fr_p3,
            ar_fr_p2,
            pop_fr_p4,
            pop_fr_p3,
            pop_fr_p2,
            ed_fr_p4,
            ed_fr_p3,
            ed_fr_p2,
            ar_frp3p4p,
            popfrp3p4p,
            ed_fr_p3p4,
            edfrp3p4p,
            ST_AsText(ST_SnapToGrid(geometry, 0.001)) as geom_wkt  -- Adjust 0.001 for your desired precision
        FROM dataset
    ''')
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return jsonify(data)

# Function to fetch project information from Jupyter Dash
def fetch_project_info():
    try:
        response = requests.get('http://127.0.0.1:8081/project-info')  # Replace with your Jupyter Dash URL
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching project info: {e}")
        return {}

# Endpoint to render homepage with project description and team member information
@app.route('/homepage')
def homepage():
    project_info = fetch_project_info()
    return render_template('homepage.html', project_info=project_info)

if __name__ == '__main__':
    app.run(debug=True, port=5000, use_reloader=False)
